In [2]:
from langchain_groq import ChatGroq

In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature = 0,
    groq_api_key = #Your Groq Api Key,
    model_name = "llama-3.1-70b-versatile"
)

In [11]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-48889")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Software Engineer II

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer IIShanghai, China MainlandBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does 

In [12]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input = {"page_data": page_data})
print(res.content)


```json
{
  "role": "Front-End Engineer II",
  "experience": "3+ years of professional experience in front-end development",
  "skills": [
    "React",
    "JavaScript/TypeScript",
    "HTML",
    "CSS",
    "WeChat Mini Program development",
    "WeChat SDK",
    "WXML",
    "WXSS",
    "Git",
    "Agile methodologies",
    "SDLC"
  ],
  "description": "We are seeking a motivated and skilled Front-End Engineer II to join our dynamic team and contribute to the development of high-quality, user-centric applications."
}
```


In [13]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Front-End Engineer II',
 'experience': '3+ years of professional experience in front-end development',
 'skills': ['React',
  'JavaScript/TypeScript',
  'HTML',
  'CSS',
  'WeChat Mini Program development',
  'WeChat SDK',
  'WXML',
  'WXSS',
  'Git',
  'Agile methodologies',
  'SDLC'],
 'description': 'We are seeking a motivated and skilled Front-End Engineer II to join our dynamic team and contribute to the development of high-quality, user-centric applications.'}

In [14]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [15]:
import chromadb
import uuid
client = chromadb.PersistentClient("vectorstore")
collection = client.get_or_create_collection(name = "portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents = row["Techstack"],
                       metadatas = {"links": row["Links"]},
                       ids = [str(uuid.uuid4())])

In [16]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}],
 [{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/kotlin-android-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ko

In [9]:
job = json_res
job['skills']

['React',
 'JavaScript/TypeScript',
 'HTML',
 'CSS',
 'WeChat Mini Program development',
 'WeChat SDK',
 'WXML',
 'WXSS',
 'Git',
 'Agile methodologies',
 'SDLC']

In [17]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Steve, a business development executive at Ollyo. Ollyo is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Steve
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Ollyo's portfolio: {link_list}
        Remember you are Steve, BDE at Ollyo. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Front-End Engineer II for Your Business Needs

Dear Hiring Manager,

I came across your job posting for a Front-End Engineer II and was impressed by the role's requirements. As a Business Development Executive at Ollyo, an AI & Software Consulting company, I'd like to introduce our team's capabilities in fulfilling your front-end development needs.

With over 3+ years of professional experience in front-end development, our experts possess the required skills, including React, JavaScript/TypeScript, HTML, CSS, WeChat Mini Program development, WeChat SDK, WXML, WXSS, Git, Agile methodologies, and SDLC. Our team is well-versed in developing high-quality, user-centric applications that meet the latest industry standards.

Our portfolio showcases our expertise in front-end development, particularly in React and TypeScript. You can view some of our notable projects at:
https://example.com/react-portfolio
https://example.com/react-native-portfolio
https://example.com/typescri